### TGB library installation 

First creat a pytion environment using pyenv or venv \
Install TGB library using pip by running *pip install py-tgb* \
Add the following dependecies: \
*pytorch == 2.0.0* \
*torch-geometric == 2.3.0* \
*torch-scatter==2.1.1* \
*torch-sparse==0.6.17* \
*torch-spline-conv==1.2.2* \
*pandas==1.5.3* \
*clint==0.5.1* 



In [1]:
from tgb.linkproppred.dataset_pyg import PyGLinkPropPredDataset
from tgb.nodeproppred.dataset_pyg import PyGNodePropPredDataset

/home/houssem.souid/.pyenv/versions/3.10.13/envs/tgb_trial/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
name = "tgbl-wiki"

In [3]:
tgbl_wiki = PyGLinkPropPredDataset(name=name, root="datasets")

raw file found, skipping download
Dataset directory is  /home/houssem.souid/tgb_trial/TGB-main/tgb/datasets/tgbl_wiki
loading processed file


In [4]:
type(tgbl_wiki)

tgb.linkproppred.dataset_pyg.PyGLinkPropPredDataset

Accessing the train, test, val split

In [95]:
train_mask = tgbl_wiki.train_mask
val_mask = tgbl_wiki.val_mask
test_mask = tgbl_wiki.test_mask


### Run experiment using TGN for link prediction

Before running the experiments you need to clone the [repository]{https://github.com/shenyangHuang/TGB/tree/main} and install the project

In [96]:
!python examples/linkproppred/tgbl-wiki/tgn.py --data "tgbl-wiki" --num_run 1 --num_epoch 5 --seed 1

python: can't open file '/home/houssem.souid/brainiac-1-temporal/notebooks/examples/linkproppred/tgbl-wiki/tgn.py': [Errno 2] No such file or directory


### Getting pyg temporal data object from tgb


The library returns a torch_geometric.data.temporal.TemporalData [type](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/data/temporal.html) object using *get_TemporalData()*  method.



In [5]:
temporal_data = tgbl_wiki.get_TemporalData()

In [6]:
type(temporal_data)

torch_geometric.data.temporal.TemporalData

In [7]:
type(tgbl_wiki)

tgb.linkproppred.dataset_pyg.PyGLinkPropPredDataset

Check node features

In [8]:
tgbl_wiki.node_feat is None

True

In [9]:
#load a dataset that has node features
name = "tgbl-flight"
tgbl_flight = PyGLinkPropPredDataset(name=name, root="datasets")

raw file found, skipping download
Dataset directory is  /home/houssem.souid/tgb_trial/TGB-main/tgb/datasets/tgbl_flight
loading processed file


In [10]:
tgbl_flight.node_feat is None

False

In [11]:
tgbl_flight_temporal_data= tgbl_flight.get_TemporalData()

The node features aren't transmitted to the [TemporalData](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/data/temporal.html) as it's not handled by this class

In [12]:
tgbl_flight_temporal_data

TemporalData(src=[67169570], dst=[67169570], t=[67169570], msg=[67169570, 16], y=[67169570])

### Getting torch_geometric_temporal from TGB

Inspired by [tigger2tgt](https://code.euranova.eu/rd/bishop/brainiac-1-temporal/-/blob/main/sota_paper_implementation/convertors/tigger2tgt.py) we can suggest the following transformation of tgt data graph from TemporalData. As the TemporalData object doesn't contain the node features contained in the tgb graph it can be modified from the tgb one.

In [17]:
from torch_geometric_temporal import dynamic_graph_temporal_signal
from tgb.linkproppred.dataset_pyg import PyGLinkPropPredDataset
from tgb.nodeproppred.dataset_pyg import PyGNodePropPredDataset
from torch_geometric.data import TemporalData
import numpy as np
from typing import Union
def convert_temp_data_to_tgt(graph_data: Union[TemporalData, PyGLinkPropPredDataset, PyGNodePropPredDataset]) -> dynamic_graph_temporal_signal.DynamicGraphTemporalSignal:
    """
    graph_data (Union[TemporalData, PyGLinkPropPredDataset, PyGNodePropPredDataset]):Temporal graph dataset

    returns: DynamicGraphTemporalSignal dataset instance
    """

    edge_indices = []
    edge_weights = []
    targets = []
    features = []
    #check type of dataset
    if isinstance(graph_data ,Union[PyGLinkPropPredDataset, PyGNodePropPredDataset ] ):
        temporal_data = graph_data.get_TemporalData()
        if graph_data.node_feat is not None:
            features = np.array(graph_data.node_feat)
    else:
        temporal_data = graph_data        
    nb_nodes = max( int(max( temporal_data.src.max() , temporal_data.dst.max() )), 0)
    #loop over snapshots
    for snapshot in temporal_data:

        edges = []
        edges.append([int(snapshot.src), int(snapshot.dst)])

        edge_indices.append(np.array(edges).T)
        edge_weights.append(np.ones(edge_indices[-1].shape[0]))
       #add features if needed
        if np.any(np.array(graph_data.node_feat) == None):
            features.append(np.ones(nb_nodes).reshape([nb_nodes, 1]))
        if snapshot.y is not None:
            targets.append(snapshot.y)
        else: 
            targets.append(np.ones(nb_nodes))
    
    
    tgt = dynamic_graph_temporal_signal.DynamicGraphTemporalSignal(
            edge_indices, edge_weights, features, targets
        )
    return tgt

Run on TGB datasets

In [18]:
#without node feat
convert_temp_data_to_tgt(tgbl_wiki)

In [19]:
#with node feat
convert_temp_data_to_tgt(tgbl_flight)

In [ ]:
import torch
from torch_geometric.data import TemporalData
#Try with a simple TemporalData
temporal_data_1 = TemporalData(
    src=torch.tensor([0, 1]),
    dst=torch.tensor([1, 2]),
    y=torch.tensor([0, 1])
)

tgt_1 = convert_temp_data_to_tgt(temporal_data_1)

# check expected edge indices
assert torch.equal(torch.tensor(tgt_1.edge_indices).squeeze() , torch.tensor([[0, 1], [1, 2]]))